In [1]:
import base64
import re
import sqlite3
import operator
from bs4 import BeautifulSoup, Comment
from dateutil.parser import parse

In [2]:
raw_conn = sqlite3.connect('raw_data/mojim.db')
raw_conn.row_factory = sqlite3.Row

In [3]:
parsed_conn = sqlite3.connect('parsed_lyrics.db')

In [4]:
parsed_conn.executescript('DROP TABLE IF EXISTS albums; DROP TABLE IF EXISTS songs;')

In [5]:
parsed_conn.executescript('''\
CREATE TABLE IF NOT EXISTS albums (
    href             TEXT PRIMARY KEY,
    title            TEXT,
    singer           TEXT,
    year             TEXT,
    lang             TEXT
);

CREATE TABLE IF NOT EXISTS songs (
    href             TEXT PRIMARY KEY,
    album_href       TEXT,
    album_title      TEXT,
    album_lang       TEXT,
    singer           TEXT,
    year             INTEGER,
    song             TEXT,
    song_writer      TEXT,
    lyrics_writer    TEXT,
    lyrics           TEXT,
    FOREIGN KEY(album_href) REFERENCES albums(href)
);
''')

### Process Album

```
href TEXT PRIMARY KEY, name TEXT, title TEXT, meta TEXT, songs TEXT, lyrics_raw TEXT
```

in base64

In [6]:
def b64_to_utf8(col):
    return base64.b64decode(col).decode(encoding='utf8')

In [7]:
def parse_album_info(soup):
    album_re = re.compile(r'【\s+(.*?)\s+】')
    album_info = album_re.findall(soup.find('h2').text)
    if len(album_info) == 3:
        title, lang, date = album_info
        release_year = parse(date, yearfirst=True).year
    elif len(album_info) == 2:
        title, lang = album_info
        release_year = None
    return title, lang, release_year

In [8]:
def filter_lyrics(lyric_strings):
    lyric_writer = None
    song_writer = None
    filtered_lyrics = []
    for line in lyric_strings:
        if line.startswith('['):
            # dynamic lyrics
            continue
        elif line.startswith('更多更詳盡歌詞') or '魔鏡歌詞網' in line:
            continue
        elif '****' in line or '----' in line or 'CDATA' in line:
            continue
        elif '作詞' in line:
            lyric_writer = line
        elif '作曲' in line:
            song_writer = line
        elif '編曲' in line:
            continue
        else:
            filtered_lyrics.append(line)
    return song_writer, lyric_writer, '\n'.join(filtered_lyrics)

In [9]:
def parse_lyric_content(dl):
    elems = dl.findChildren(name=re.compile(r'(dd|dt)'))
    for dt, dd in zip(*[iter(elems)]*2):
        try:
            song_name = dt.find(lambda tag: tag.name == 'a' and tag.has_attr('href')).text
            song_href = dt.find('a', attrs={'href': re.compile('^/twy')}).attrs['href']
        except Exception:
            continue
        try:
            unwanted_tags = ['hr', 'ol', 'ul', 'b', 'script']
            for tag in unwanted_tags:
                for dom in dd.select(tag):
                    dom.decompose()
            song_writer, lyric_writer, lyrics = filter_lyrics(dd.strings)
        except:
            song_writer, lyric_writer, lyrics = None, None, None
        yield song_href, song_name, song_writer, lyric_writer, lyrics
    
def parse_lyric_table(soup):
    lyric_table = soup.find('table')
    dls = lyric_table.find_all(lambda dom: dom.name == 'dl' and dom.previous_element == '專輯歌詞')
    record_songs = set()
    for dl in dls:
        for song_info in parse_lyric_content(dl):
            if song_info[0] in record_songs:
                continue
            record_songs.add(song_info[0])
            yield song_info

### Update for full database

In [10]:
from IPython.html import widgets
from IPython.display import display as ipydisplay

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [11]:
cur = raw_conn.execute(
    'SELECT href, name, title, meta, songs, lyrics_raw '
    'FROM mojim '
    # 'WHERE name = "QUFST04="'
)

In [12]:
# row = next(cur)
# lyrics_raw = b64_to_utf8(row['lyrics_raw'])
# soup = BeautifulSoup(lyrics_raw, 'html.parser')
# list(parse_lyric_table(soup))

In [13]:
# meta = BeautifulSoup(b64_to_utf8(row['meta']))
# # remove producer company
# [d.decompose() for d in meta.span.select('font')]
# meta.span.text

In [14]:
!date

Sat May  2 00:34:28 CST 2015


In [ ]:
w = widgets.IntProgress()
w.max = tuple(raw_conn.execute('SELECT COUNT(*) FROM mojim').fetchone())[0]
ipydisplay(w)

for row in cur:
    singer = b64_to_utf8(row['name'])
    w.description = singer
    w.value += 1
    try:
        lyrics_raw = b64_to_utf8(row['lyrics_raw'])
        soup = BeautifulSoup(lyrics_raw, 'html.parser')
        title, lang, year = parse_album_info(soup)
        song_lists = list(parse_lyric_table(soup))
    except Exception as e:
        print('Processing %s album %s failed with %r' % (singer, row['href'], e))
        continue
    cur = parsed_conn.cursor()
    cur.execute(
        'INSERT OR IGNORE INTO albums '
        '(href, title, singer, year, lang)'
        'VALUES (?, ?, ?, ?, ?)',
        (row['href'], title, singer, year, lang)
    )
    cur.executemany(
        'INSERT OR IGNORE INTO songs ('
        'href, album_href, album_title, album_lang, '
        'singer, year, song, song_writer, lyrics_writer, '
        'lyrics )'
        'VALUES (?, ?, ?, ?, ?, ?, ?, ?, ? ,?)',
        (
            (song_href, row['href'], title, lang,
             singer, year, song_name, song_writer, lyric_writer,
             lyrics) 
            for song_href, song_name, song_writer, lyric_writer, lyrics in song_lists
        )
    )
    cur.close()
    # parsed_conn.commit()
    # if w.value == 1000:
    #     break

In [ ]:
!date